This WIP notebook is for SOU's Environmental Science camera trap project using Megadetector. This notebook is to process videos and will likely also process still images, we are evaluating how long it will take to process a very large SD card.

In [ ]:
import os
import json
import shutil
import subprocess
from pathlib import Path

import cv2
import torch

print("CUDA available:", torch.cuda.is_available())
#TODO- Improve CUDA GPU or CPU detection and usage
#TODO Megadetector Install instructions and README in github
#TODO workflow explanation for data, paths, permissions, outputs
#TODO- instructions for one video then batch
#TODO videoplayer inline in notebook to view results
#TODO detection videos are all mp4s from AVI, MOV, MP4- does this matter to client? does it matter at all?

In [ ]:
# Root of MegaDetector repo
MEGADETECTOR_ROOT = Path("/home/jupyter-bernie/MegaDetector")
#TODO remove hardcoded paths
# Root directory containing videos (recursive)
DATA_DIR = Path("/home/jupyter-bernie/6June2025")
# This notebook presumes both Megadetector and DATA_DIR folders are side by side (in this case in Jupyter Hub)
# For NAIRR, will put data in a shared mounted volume on Jetstream and figure out permissions from there
# Output directory (MUST be outside DATA_DIR)
OUTPUT_DIR = Path("./megadetector_outputs_batch")
OUTPUT_DIR.mkdir(exist_ok=True)

# Model
MODEL_NAME = "MDV5A"
# This is 'classic Dan Morris' MEGADETECTOR 5
# Detection parameters
ANIMAL_CATEGORY_ID = 1
CONF_THRESH = 0.5


In [ ]:
assert DATA_DIR.exists(), "DATA_DIR not found"
assert (MEGADETECTOR_ROOT / "megadetector/detection/run_detector_batch.py").exists()


In [ ]:
def find_all_videos(root):
    exts = (".mp4", ".mov", ".avi", ".MP4", ".MOV", ".AVI")
    videos = []
    for r, _, files in os.walk(root):
        for f in files:
            if f.endswith(exts):
                videos.append(Path(r) / f)
    return sorted(videos)

all_videos = find_all_videos(DATA_DIR)
print(f"Found {len(all_videos)} videos")

for v in all_videos[:20]:
    print(v)


In [ ]:
def resolve_frame_path(frames_dir, file_field):
    p = Path(file_field)
    if p.is_absolute() or p.exists():
        return p
    return frames_dir / p.name


In [ ]:
def video_has_any_animal(images, conf_thresh=0.0):
    for img in images:
        for det in img.get("detections", []):
            if int(det["category"]) == ANIMAL_CATEGORY_ID and det["conf"] >= conf_thresh:
                return True
    return False


In [ ]:
import time

batch_start = time.perf_counter()

videos_processed = 0
videos_with_animals = 0
videos_skipped = 0
videos_with_animals_list = []
videos_skipped_list = []


for video_path in all_videos:
#for video_path in all_videos[:3]:  # limit while testing to 3 videos, replace this line and comment line above

    print("\n==============================")
    print("Processing:", video_path)

    # ----------------------------------------
    # Per-video output paths
    # ----------------------------------------
    video_stem = video_path.stem
    video_out_dir = OUTPUT_DIR / video_stem

    frames_dir = video_out_dir / "frames"
    json_path = video_out_dir / "detections.json"
    out_video = video_out_dir / f"{video_stem}_detected.mp4"

    video_out_dir.mkdir(parents=True, exist_ok=True)

    print("Frames:", frames_dir)
    print("JSON:", json_path)
    print("Out:", out_video)

    # ----------------------------------------
    # Clean frames dir
    # ----------------------------------------
    if frames_dir.exists():
        shutil.rmtree(frames_dir)
    frames_dir.mkdir()

    # ----------------------------------------
    # Extract frames
    # ----------------------------------------
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(
            str(frames_dir / f"frame_{frame_idx:05d}.jpg"),
            frame
        )
        frame_idx += 1

    cap.release()
    print(f"Extracted {frame_idx} frames @ {fps:.2f} FPS")

    if frame_idx == 0:
        print("No frames, skipping")
        continue

    # ----------------------------------------
    # Run MegaDetector
    # ----------------------------------------
    cmd = [
        "python",
        str(MEGADETECTOR_ROOT / "megadetector/detection/run_detector_batch.py"),
        MODEL_NAME,
        str(frames_dir),
        str(json_path),
    ]

    subprocess.run(cmd, check=True)

    # ----------------------------------------
    # Load detections
    # ----------------------------------------
    with open(json_path) as f:
        md = json.load(f)

    images = md.get("images", [])
    print("Images in JSON:", len(images))

    if len(images) != frame_idx:
        print("Frame/JSON mismatch, skipping")
        shutil.rmtree(frames_dir)
        continue

# --------------------------------------------------
# NEW: Skip videos with NO animal detections
# --------------------------------------------------
    if not video_has_any_animal(images, CONF_THRESH):
        print(" No animal detections in entire video — skipping output")
        shutil.rmtree(frames_dir)
        videos_skipped += 1
        videos_processed += 1
        videos_skipped_list.append(video_path.name)
        continue


    # ----------------------------------------
    # Initialize output video
    # ----------------------------------------
    first_frame_path = resolve_frame_path(frames_dir, images[0]["file"])
    first_frame = cv2.imread(str(first_frame_path))

    if first_frame is None:
        print("Cannot read first frame, skipping")
        continue

    h, w, _ = first_frame.shape

    out = cv2.VideoWriter(
        str(out_video),
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w, h),
    )

    # ----------------------------------------
    # Write full video (boxes on animals only)
    # ----------------------------------------
    written = 0

    for img in images:
        frame_path = resolve_frame_path(frames_dir, img["file"])
        frame = cv2.imread(str(frame_path))
        if frame is None:
            continue

        for det in img.get("detections", []):
            if int(det["category"]) != ANIMAL_CATEGORY_ID:
                continue
            if det["conf"] < CONF_THRESH:
                continue

            x, y, bw, bh = det["bbox"]
            x1 = int(x * w)
            y1 = int(y * h)
            x2 = int((x + bw) * w)
            y2 = int((y + bh) * h)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"{det['conf']:.2f}",
                (x1, max(y1 - 5, 10)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
            )

        out.write(frame)
        written += 1

    out.release()
    print(f"Wrote {written} frames to {out_video}")
    videos_with_animals += 1
    videos_processed += 1
    videos_with_animals_list.append(video_path.name)
    # ----------------------------------------
    # Cleanup frames
    # ----------------------------------------
    shutil.rmtree(frames_dir)


In [ ]:
import time

batch_end = time.perf_counter()
elapsed = batch_end - batch_start

print("\n==============================")
print("Batch processing summary")
print("==============================")
print(f"Total videos found     : {len(all_videos)}")
print(f"Videos processed       : {videos_processed}")
print(f"Videos with animals    : {videos_with_animals}")
print(f"Videos skipped (empty) : {videos_skipped}")
print(f"Total elapsed time     : {elapsed:.2f} seconds")
print(f"Average per video      : {elapsed / max(videos_processed, 1):.2f} seconds")

print("\n Videos WITH animal detections:")
if videos_with_animals_list:
    for v in videos_with_animals_list:
        print("  ✔", v)
else:
    print("  (none)")

print("Videos with NO animal detections:")
if videos_skipped_list:
    for v in videos_skipped_list:
        print("  ✖", v)
else:
    print("  (none)")

In [ ]:
#TODO- separate out extractions and whatnot into different cells
#TODO- more cleanup? Delete everything but videos with detections? 
# ES only wants videos with detections to open in TimeLapse